In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import TreebankWordTokenizer as twt

In [2]:
from scipy.spatial.distance import cdist

In [3]:
import re
import itertools
import math

In [4]:
import os.path
from os import path

In [5]:
from ast import literal_eval


**Read a Data set**

In [6]:
data = pd.read_excel('bio_df_final.xlsx')

print (data.shape)


(11871, 5)


In [7]:
data.head(3)

,sid,oidx,sentences,neg_scopes,spec_scopes
0,S1.1,0,Induction of NF-KB during monocyte differentia...,[],[]
1,S1.2,1,The production of human immunodeficiency virus...,[],[]
2,S1.3,2,Electrophoretic mobility shift assays and Sout...,[],[]


In [8]:
data['neg_scopes'] = data['neg_scopes'].apply(lambda x: literal_eval(x))
data['spec_scopes'] = data['spec_scopes'].apply(lambda x: literal_eval(x))

In [9]:
data.index

RangeIndex(start=0, stop=11871, step=1)

In [10]:
scope = "not in the S1 and S1' subsites of cathepsin D"
scope_new = ' '.join(twt().tokenize(scope))

len_scope = len(twt().tokenize(scope_new))

bigs = "From information available in the literature on the substrate \
specificity of the three enzymes, it is clear that a cysteine is not accepted \
in any of the S2, S1, S1', and S2' subsites of cathepsin B and L, and not in the S1 and S1' subsites of cathepsin D."
bigs_tokens = twt().tokenize(bigs)
bigs_new = ' '.join(bigs_tokens)

print (bigs_new)
print (scope_new)
remains = bigs_new.replace(scope_new,'')
len_remains= len(twt().tokenize(remains))

print (remains, len_remains)

label_scope = []
for i in range(len_scope):
    if i==0:
        label_scope.append('B-Negation')
    else:
        label_scope.append('I-Negation')
        
bio = ' '.join(label_scope)
print (bio)

label_remains = []
for i in range(len_remains):
    label_remains.append('O')
    
others = ' '.join(label_remains)
print (others)

temps = bigs_new.replace(scope_new, bio)
print (temps)

temps_tokens = twt().tokenize(temps)
annotated_string = ['O' if (elem not in ['B-Negation', 'I-Negation']) else elem for elem in temps_tokens]
#print (annotated_string)

# annotated_string = temps.replace(remains.strip(), others)

ann_df = pd.DataFrame()
ann_df['token'] = bigs_tokens
ann_df['tag'] = annotated_string
ann_df

From information available in the literature on the substrate specificity of the three enzymes , it is clear that a cysteine is not accepted in any of the S2 , S1 , S1 ' , and S2 ' subsites of cathepsin B and L , and not in the S1 and S1 ' subsites of cathepsin D .
not in the S1 and S1 ' subsites of cathepsin D
From information available in the literature on the substrate specificity of the three enzymes , it is clear that a cysteine is not accepted in any of the S2 , S1 , S1 ' , and S2 ' subsites of cathepsin B and L , and  . 47
B-Negation I-Negation I-Negation I-Negation I-Negation I-Negation I-Negation I-Negation I-Negation I-Negation I-Negation
O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O
From information available in the literature on the substrate specificity of the three enzymes , it is clear that a cysteine is not accepted in any of the S2 , S1 , S1 ' , and S2 ' subsites of cathepsin B and L , and B-Negation I-Negation I-Negation

,token,tag
0,From,O
1,information,O
2,available,O
3,in,O
4,the,O
5,literature,O
6,on,O
7,the,O
8,substrate,O
9,specificity,O


In [11]:
def convert_bio(sent,scope_list):
    bigs_tokens = twt().tokenize(sent)
    bigs_new = ' '.join(bigs_tokens)
    
    if (len(scope_list) == 0):
        annotated_string = ['O' for elem in bigs_tokens]
    else:
        for scope in scope_list:
            scope_new = ' '.join(twt().tokenize(scope))
            len_scope = len(twt().tokenize(scope_new))

#             print (bigs_new)
#             print (scope_new)
            remains = bigs_new.replace(scope_new,'')
            len_remains= len(twt().tokenize(remains))

#            print (remains, len_remains)

            label_scope = []
            for i in range(len_scope):
                if i==0:
                    label_scope.append('B-Negation')
                else:
                    label_scope.append('I-Negation')

            bio = ' '.join(label_scope)
#            print (bio)

            label_remains = []
            for i in range(len_remains):
                label_remains.append('O')

            others = ' '.join(label_remains)
#            print (others)

            temps = bigs_new.replace(scope_new, bio)
#            print (scope_new)
            #print (bio)
#            print (bigs_new)    
#            print (temps)
            bigs_new = temps
            temps_tokens = twt().tokenize(temps)
            annotated_string = ['O' if (elem not in ['B-Negation', 'I-Negation']) else elem for elem in temps_tokens]
#            print (temps_tokens)
#            print (annotated_string)

            # annotated_string = temps.replace(remains.strip(), others)

    ann_df = pd.DataFrame()
    ann_df['token'] = twt().tokenize(sent)
    ann_df['tag'] = annotated_string
    
    return ann_df

In [13]:
s1 = "not in the S1 and S1' subsites of cathepsin D"
s2 = "a cysteine is not accepted in any of the S2, S1, S1'"
sbig = "From information available in the literature on the substrate \
specificity of the three enzymes, it is clear that a cysteine is not accepted \
in any of the S2, S1, S1', and S2' subsites of cathepsin B and L, and not in the S1 and S1' subsites of cathepsin D."

#convert_bio("i dont (know), i love you too.", ['i dont', 'you too'])
convert_bio(sbig, [s1, s2])

,token,tag
0,From,O
1,information,O
2,available,O
3,in,O
4,the,O
5,literature,O
6,on,O
7,the,O
8,substrate,O
9,specificity,O


In [14]:
data.head()

,sid,oidx,sentences,neg_scopes,spec_scopes
0,S1.1,0,Induction of NF-KB during monocyte differentia...,[],[]
1,S1.2,1,The production of human immunodeficiency virus...,[],[]
2,S1.3,2,Electrophoretic mobility shift assays and Sout...,[],[]
3,S1.4,3,"PMA treatment, and not retinoic acid treatment...",[not retinoic acid treatment of the U937 cells],[]
4,S1.5,4,In nuclear extracts from monocytes or macropha...,[],[]


In [15]:
#data['annotations'] = data.apply(lambda x: convert_bio(x.sentences, x.neg_scopes), axis=1)

In [16]:
data.head()

,sid,oidx,sentences,neg_scopes,spec_scopes
0,S1.1,0,Induction of NF-KB during monocyte differentia...,[],[]
1,S1.2,1,The production of human immunodeficiency virus...,[],[]
2,S1.3,2,Electrophoretic mobility shift assays and Sout...,[],[]
3,S1.4,3,"PMA treatment, and not retinoic acid treatment...",[not retinoic acid treatment of the U937 cells],[]
4,S1.5,4,In nuclear extracts from monocytes or macropha...,[],[]


In [17]:
snts = data['sentences'].to_list()
ngs = data['neg_scopes'].to_list()
ngs

negation_bio = []
for s,n in zip(snts,ngs):
    negation_bio.append((convert_bio(s,n)))

In [18]:
negation_bio

[              token tag
 0         Induction   O
 1                of   O
 2             NF-KB   O
 3            during   O
 4          monocyte   O
 5   differentiation   O
 6                by   O
 7               HIV   O
 8              type   O
 9                 1   O
 10        infection   O
 11                .   O,                token tag
 0                The   O
 1         production   O
 2                 of   O
 3              human   O
 4   immunodeficiency   O
 5              virus   O
 6               type   O
 7                  1   O
 8                  (   O
 9              HIV-1   O
 10                 )   O
 11           progeny   O
 12               was   O
 13          followed   O
 14                in   O
 15               the   O
 16              U937   O
 17      promonocytic   O
 18              cell   O
 19              line   O
 20             after   O
 21       stimulation   O
 22            either   O
 23              with   O
 24          retinoic   O

In [19]:
data_concat = pd.concat(negation_bio,keys = data.sid.to_list())
data_concat.reset_index(inplace=True)

In [20]:
data_concat.iloc[50:100,:]

,level_0,level_1,token,tag
50,S1.3,1,mobility,O
51,S1.3,2,shift,O
52,S1.3,3,assays,O
53,S1.3,4,and,O
54,S1.3,5,Southwestern,O
55,S1.3,6,blotting,O
56,S1.3,7,experiments,O
57,S1.3,8,were,O
58,S1.3,9,used,O
59,S1.3,10,to,O


In [21]:
def train_test_dev_split(pmid, p_train = 0.7, p_test = 0.15, r1 = 42, r2 = 100):
    """
    split the id list into trian test dev list

    pmid: list list of id for this func
    p_train: train set per
    p_test: test set per
    r1: random state for first split
    r2: random state for second split

    return:
        train_pmid: list for train
        test_pmid: list for test
        dev_pmid: list for dev
    """
    train_pmid, other = train_test_split(pmid, test_size = (1 - p_train), random_state = r1)
    test_pmid, dev_pmid = train_test_split(other, test_size = p_test/(1 - p_train), random_state = r2)
    return train_pmid, test_pmid, dev_pmid

# This function just randomly splits the data into training, dev, and test

In [22]:
train_data, test_data, dev_data = train_test_dev_split(data)

In [23]:
#C:\Users\Sri\Google Drive\Data Science\Python\Data_formats\bio_xml_parse\bio_output\negations
def output_bio(name, sample):
    out_file = name + '.txt'
    for i in sample:
    #     print(i)
        temp = data_concat[data_concat.level_0 == i][['token','tag']]
    #     temp.word = temp.word.apply(lambda x: x.lower())
        temp.to_csv(out_file, header = None, index = None,
                               sep = '\t', mode = 'a')
    #     print(temp)
        # add a new line after each sentence
        with open(out_file, 'a') as f:
            f.write('\n')
    #     print('\n')
    return 'done'

In [ ]:
output_bio('output', data['sid'].to_list())

In [ ]:
#extract_bio(data, ['neg_scopes'])

In [ ]:
data_result = extract_bio(data, ['neg_scopes'])

In [ ]:
len(data_result)

In [ ]:
data_result

In [ ]:
data_concat = pd.concat(data_result, keys = data.id.to_list())
data_concat.reset_index(inplace=True)
data_concat.head()

**Split the data into train, dev, test**

In [ ]:
#train_data, test_data, dev_data = train_test_dev_split(data)
# Uncomment above & comment below, if the final data to be bio tagged is read directly and has not already been split.
train_data = new_train
dev_data = new_dev
test_data = new_test

In [ ]:
id_ls = list(data_concat.level_0.unique())

In [ ]:
len(id_ls)

**Output the bio tagged data**

In [ ]:
# Enter the correct path below, where the BIO files need to be written

In [ ]:
def output_bio(name, sample):
      
    out_file = 'data/ner/new_golden/' + name +'_actor.txt'
    if(path.exists(out_file)):
        print ('BIO files already exist')
        return 'done'
    else:
        for i in sample:
        #     print(i)
            temp = data_concat[data_concat.level_0 == i][['word','pos','actor']]
        #     temp.word = temp.word.apply(lambda x: x.lower())
            temp.to_csv(out_file, header = None, index = None,
                                   sep = '\t', mode = 'a')
        #     print(temp)
            # add a new line after each sentence
            with open(out_file, 'a') as f:
                f.write('\n')
        #     print('\n')
    return 'done'


In [ ]:
# below code writes the output into text files with "name_actor": Ex: "train_actor".txt 
# dont run them twice, since the files will be appended

In [ ]:
output_bio('train', train_data['id'].to_list())

In [ ]:
output_bio('dev', dev_data['id'].to_list())

In [ ]:
output_bio('test', test_data['id'].to_list())